In [240]:
'''
수정을 자주 할 것 같으니 최대한 잘게 쪼개서 적어보기
'''
import warnings
warnings.filterwarnings(action='ignore') # 경고 없애기

import pandas as pd
import numpy as np
import time
from PIL import Image

In [241]:
# 한글, -기호 깨짐 방지

import matplotlib.pyplot as plt

plt.rc("font", family="Malgun Gothic")
plt.rc("axes", unicode_minus=False)

In [242]:
# 그래프 선명하게 보여주기

%config InlineBackend.figure_format = 'retina'

In [243]:
# 데이터 불러오기

df1 = pd.read_csv("TFT_Challenger_MatchData.csv")
df2 = pd.read_csv("TFT_GrandMaster_MatchData.csv")
df3 = pd.read_csv("TFT_Master_MatchData.csv")
df4 = pd.read_csv("TFT_Diamond_MatchData.csv")
df5 = pd.read_csv("TFT_Platinum_MatchData.csv")
df_champ = pd.read_csv("TFT_Champion_CurrentVersion.csv")
df_item = pd.read_csv("TFT_Item_CurrentVersion.csv")

In [244]:
'''
어느 combination(시너지)가 승리(4위 이상) 확률이 높은지 보기 위해서 컬럼을 합쳐야 할 수 있으니
이후에 구분하기 위해 티어 컬럼 추가
'''
df1['Tier'] = 'Challenger'
df2['Tier'] = 'GrandMaster'
df3['Tier'] = 'Master'
df4['Tier'] = 'Diamond'
df5['Tier'] = 'Platinum'

In [245]:
# 결측치 제거하기

df1 = df1.dropna()
df2 = df2.dropna()
df3 = df3.dropna()
df4 = df4.dropna()
df5 = df5.dropna()
df_champ = df_champ.dropna()
df_item = df_item.dropna()

In [246]:
# Round가 충분히 진행되지 않은 상태에서 탈락한 경우(20라운드 미만으로 가정) 이상치로 지정
# df1~df5 에서 lastRound가 20 이상인 경우만 남기기

dfs = [df1, df2, df3, df4, df5]

for i, df in enumerate(dfs):
    df = df[df['lastRound'] >= 20] 
    dfs[i] = df

df1, df2, df3, df4, df5 = dfs

In [247]:
# 결측치 하나씩 확인하기 

df1.isnull().sum()

gameId            0
gameDuration      0
level             0
lastRound         0
Ranked            0
ingameDuration    0
combination       0
champion          0
Tier              0
dtype: int64

In [248]:
# df1~df5 한번에 결측치 확인하기

dfs = [df1, df2, df3, df4, df5]
null_counts = {f"df{i+1}": df.isnull().sum() for i, df in enumerate(dfs)} # 딕셔너리 컴프리헨션
null_counts

{'df1': gameId            0
 gameDuration      0
 level             0
 lastRound         0
 Ranked            0
 ingameDuration    0
 combination       0
 champion          0
 Tier              0
 dtype: int64,
 'df2': gameId            0
 gameDuration      0
 level             0
 lastRound         0
 Ranked            0
 ingameDuration    0
 combination       0
 champion          0
 Tier              0
 dtype: int64,
 'df3': gameId            0
 gameDuration      0
 level             0
 lastRound         0
 Ranked            0
 ingameDuration    0
 combination       0
 champion          0
 Tier              0
 dtype: int64,
 'df4': gameId            0
 gameDuration      0
 level             0
 lastRound         0
 Ranked            0
 ingameDuration    0
 combination       0
 champion          0
 Tier              0
 dtype: int64,
 'df5': gameId            0
 gameDuration      0
 level             0
 lastRound         0
 Ranked            0
 ingameDuration    0
 combination       0
 c

In [249]:
# df5 실행 -> 이상치가 사라져 79257row 남음

df5

,gameId,gameDuration,level,lastRound,Ranked,ingameDuration,combination,champion,Tier
0,KR_4291707834,1963.905273,6,27,5,1390.165771,"{'Cybernetic': 1, 'Demolitionist': 1, 'Infiltr...","{'Ziggs': {'items': [7], 'star': 1}, 'Ashe': {...",Platinum
1,KR_4291707834,1963.905273,8,37,3,1891.282715,"{'Blaster': 1, 'Chrono': 1, 'Cybernetic': 4, '...","{'Ziggs': {'items': [24], 'star': 3}, 'Fiora':...",Platinum
2,KR_4291707834,1963.905273,6,25,7,1279.461060,"{'Blaster': 1, 'Cybernetic': 1, 'DarkStar': 2,...","{'Fiora': {'items': [1], 'star': 1}, 'Shaco': ...",Platinum
3,KR_4291707834,1963.905273,7,38,2,1955.608521,"{'DarkStar': 1, 'Protector': 2, 'Set3_Blademas...","{'Poppy': {'items': [], 'star': 2}, 'Xayah': {...",Platinum
4,KR_4291707834,1963.905273,8,38,1,1955.608521,"{'Blaster': 1, 'Chrono': 5, 'DarkStar': 3, 'Pr...","{'TwistedFate': {'items': [36, 27], 'star': 3}...",Platinum
...,...,...,...,...,...,...,...,...,...
79995,KR_4296831363,2049.954102,7,34,4,1862.439209,"{'Chrono': 2, 'Cybernetic': 1, 'Protector': 2,...","{'TwistedFate': {'items': [58], 'star': 2}, 'X...",Platinum
79996,KR_4296831363,2049.954102,8,33,5,1799.791870,"{'Chrono': 2, 'Cybernetic': 1, 'Infiltrator': ...","{'Xayah': {'items': [5], 'star': 2}, 'Yasuo': ...",Platinum
79997,KR_4296831363,2049.954102,8,38,2,2041.694946,"{'Blaster': 1, 'DarkStar': 1, 'MechPilot': 1, ...","{'Poppy': {'items': [], 'star': 2}, 'Zoe': {'i...",Platinum
79998,KR_4296831363,2049.954102,8,35,3,1911.133057,"{'Blaster': 2, 'Chrono': 1, 'Demolitionist': 1...","{'Ziggs': {'items': [44, 48], 'star': 3}, 'Mal...",Platinum


In [250]:
# champion(기물)수가 5개 이하인 경우 이상치로 처리

dfs = [df1, df2, df3, df4, df5]

dfs = [df[df['champion'].apply(lambda x: len(ast.literal_eval(x)) >= 5)] for df in dfs]

df1, df2, df3, df4, df5 = dfs

In [251]:
# df5 실행(2) -> 이상치가 사라져 79053row 남음

df5

,gameId,gameDuration,level,lastRound,Ranked,ingameDuration,combination,champion,Tier
1,KR_4291707834,1963.905273,8,37,3,1891.282715,"{'Blaster': 1, 'Chrono': 1, 'Cybernetic': 4, '...","{'Ziggs': {'items': [24], 'star': 3}, 'Fiora':...",Platinum
3,KR_4291707834,1963.905273,7,38,2,1955.608521,"{'DarkStar': 1, 'Protector': 2, 'Set3_Blademas...","{'Poppy': {'items': [], 'star': 2}, 'Xayah': {...",Platinum
4,KR_4291707834,1963.905273,8,38,1,1955.608521,"{'Blaster': 1, 'Chrono': 5, 'DarkStar': 3, 'Pr...","{'TwistedFate': {'items': [36, 27], 'star': 3}...",Platinum
6,KR_4291707834,1963.905273,6,33,4,1698.420166,"{'Protector': 1, 'Set3_Mystic': 1, 'Set3_Sorce...","{'Poppy': {'items': [], 'star': 2}, 'Zoe': {'i...",Platinum
8,KR_4291614366,2082.444824,7,33,4,1891.497559,"{'Blaster': 1, 'DarkStar': 2, 'Set3_Celestial'...","{'Graves': {'items': [], 'star': 2}, 'Poppy': ...",Platinum
...,...,...,...,...,...,...,...,...,...
79995,KR_4296831363,2049.954102,7,34,4,1862.439209,"{'Chrono': 2, 'Cybernetic': 1, 'Protector': 2,...","{'TwistedFate': {'items': [58], 'star': 2}, 'X...",Platinum
79996,KR_4296831363,2049.954102,8,33,5,1799.791870,"{'Chrono': 2, 'Cybernetic': 1, 'Infiltrator': ...","{'Xayah': {'items': [5], 'star': 2}, 'Yasuo': ...",Platinum
79997,KR_4296831363,2049.954102,8,38,2,2041.694946,"{'Blaster': 1, 'DarkStar': 1, 'MechPilot': 1, ...","{'Poppy': {'items': [], 'star': 2}, 'Zoe': {'i...",Platinum
79998,KR_4296831363,2049.954102,8,35,3,1911.133057,"{'Blaster': 2, 'Chrono': 1, 'Demolitionist': 1...","{'Ziggs': {'items': [44, 48], 'star': 3}, 'Mal...",Platinum


In [252]:
# 매치 데이터 합치기
# 리스트로 묶은 후 판다스에서 데이터프레임 합치고 인덱스 재정렬

dfs = [df1, df2, df3, df4, df5]
df_all = pd.concat(dfs, ignore_index=True)

# 경고 없애기
df_all = df_all.copy()

In [253]:
df_all

,gameId,gameDuration,level,lastRound,Ranked,ingameDuration,combination,champion,Tier
0,KR_4247538593,2142.470703,8,35,1,2134.272217,"{'DarkStar': 2, 'Protector': 4, 'Rebel': 1, 'S...","{'JarvanIV': {'items': [27], 'star': 3}, 'Sona...",Challenger
1,KR_4247538593,2142.470703,9,35,2,2134.272217,"{'Blaster': 2, 'Mercenary': 1, 'Rebel': 6, 'Se...","{'Malphite': {'items': [7], 'star': 2}, 'Yasuo...",Challenger
2,KR_4247538593,2142.470703,8,34,3,2073.459229,"{'Cybernetic': 1, 'DarkStar': 3, 'Demolitionis...","{'KaiSa': {'items': [99, 2, 23], 'star': 2}, '...",Challenger
3,KR_4247538593,2142.470703,8,33,4,1998.146729,"{'Blaster': 1, 'Cybernetic': 1, 'DarkStar': 1,...","{'KaiSa': {'items': [44, 37], 'star': 2}, 'Ann...",Challenger
4,KR_4247538593,2142.470703,9,33,5,1986.443237,"{'Blaster': 2, 'Demolitionist': 2, 'Mercenary'...","{'Ziggs': {'items': [], 'star': 1}, 'Yasuo': {...",Challenger
...,...,...,...,...,...,...,...,...,...
396107,KR_4296831363,2049.954102,7,34,4,1862.439209,"{'Chrono': 2, 'Cybernetic': 1, 'Protector': 2,...","{'TwistedFate': {'items': [58], 'star': 2}, 'X...",Platinum
396108,KR_4296831363,2049.954102,8,33,5,1799.791870,"{'Chrono': 2, 'Cybernetic': 1, 'Infiltrator': ...","{'Xayah': {'items': [5], 'star': 2}, 'Yasuo': ...",Platinum
396109,KR_4296831363,2049.954102,8,38,2,2041.694946,"{'Blaster': 1, 'DarkStar': 1, 'MechPilot': 1, ...","{'Poppy': {'items': [], 'star': 2}, 'Zoe': {'i...",Platinum
396110,KR_4296831363,2049.954102,8,35,3,1911.133057,"{'Blaster': 2, 'Chrono': 1, 'Demolitionist': 1...","{'Ziggs': {'items': [44, 48], 'star': 3}, 'Mal...",Platinum


In [254]:
# 존재하는 combination 확인 해보기
# combination이 문자열이라 딕셔너리로 변환 후 확인
import ast

all_combinations = set()  # 중복 제거를 위한 set

for comb_str in df_all['combination']:
    comb_dict = ast.literal_eval(comb_str)  # 문자열 → 딕셔너리
    all_combinations.update(comb_dict.keys())  # 키 추가

print(all_combinations)


{'Soulbound', 'Ocean', 'Mercenary', 'Mage', 'Mountain', 'SpacePirate', 'Blaster', 'StarGuardian', 'MechPilot', 'Light', 'Avatar', 'Berserker', 'Cybernetic', 'Desert', 'Metal', 'Mystic', 'Shadow', 'Sniper', 'Summoner', 'Electric', 'Druid', 'Set3_Celestial', 'Vanguard', 'Set3_Brawler', 'Predator', 'Set2_Ranger', 'Set2_Assassin', 'Rebel', 'Set2_Glacial', 'Chrono', 'DarkStar', 'Infiltrator', 'Set3_Blademaster', 'Demolitionist', 'Set3_Mystic', 'Crystal', 'Inferno', 'Set2_Blademaster', 'Valkyrie', 'Woodland', 'Celestial', 'ManaReaver', 'Wind', 'Alchemist', 'Protector', 'Poison', 'TemplateTrait', 'Set3_Void', 'Starship', 'Warden', 'Set3_Sorcerer'}


In [255]:
# combination(시너지)를 한글로 매핑 해보자

combination_map = {
    "Void": "공허",
    "MechPilot": "메카 파일럿",
    "Rebel": "반군",
    "Valkyrie": "발키리",
    "StarGuardian": "별 수호자",
    "Cybernetic": "사이버네틱",
    "Chrono": "시공간",
    "DarkStar": "암흑의 별",
    "SpacePirate": "우주 해적",
    "Celestial": "천상",
    "Blademaster": "검사",
    "ManaReaver": "마나 약탈자",
    "Sorcerer": "마법사",
    "Vanguard": "선봉대",
    "Protector": "수호자",
    "Mystic": "신비술사",
    "Brawler": "싸움꾼",
    "Mercenary": "용병",
    "Starship": "우주선",
    "Infiltrator": "잠입자",
    "Sniper": "저격수",
    "Blaster": "총잡이",
    "Demolitionist": "폭파광",
}

In [256]:

# set n_ 으로 겹치는 이름이 몇개 있네? 제거하고 한글로 바꾸기

import ast

def combination_korean(comb_str):
    comb_dict = ast.literal_eval(comb_str)
    new_dict = {}
    for k, v in comb_dict.items():
        # Set2_,Set3_ 제거
        if k.startswith("Set2_") or k.startswith("Set3_"):
            k = k.split("_")[1]
        # 한글 매핑
        k_kr = combination_map.get(k, k)
        new_dict[k_kr] = v
    return new_dict

df_all['combination'] = df_all['combination'].apply(combination_korean)

In [257]:
df_all

,gameId,gameDuration,level,lastRound,Ranked,ingameDuration,combination,champion,Tier
0,KR_4247538593,2142.470703,8,35,1,2134.272217,"{'암흑의 별': 2, '수호자': 4, '반군': 1, '천상': 3, '신비술사...","{'JarvanIV': {'items': [27], 'star': 3}, 'Sona...",Challenger
1,KR_4247538593,2142.470703,9,35,2,2134.272217,"{'총잡이': 2, '용병': 1, '반군': 6, '검사': 3, '싸움꾼': 1...","{'Malphite': {'items': [7], 'star': 2}, 'Yasuo...",Challenger
2,KR_4247538593,2142.470703,8,34,3,2073.459229,"{'사이버네틱': 1, '암흑의 별': 3, '폭파광': 1, '잠입자': 4, '...","{'KaiSa': {'items': [99, 2, 23], 'star': 2}, '...",Challenger
3,KR_4247538593,2142.470703,8,33,4,1998.146729,"{'총잡이': 1, '사이버네틱': 1, '암흑의 별': 1, '폭파광': 2, '...","{'KaiSa': {'items': [44, 37], 'star': 2}, 'Ann...",Challenger
4,KR_4247538593,2142.470703,9,33,5,1986.443237,"{'총잡이': 2, '폭파광': 2, '용병': 2, '반군': 6, '검사': 2...","{'Ziggs': {'items': [], 'star': 1}, 'Yasuo': {...",Challenger
...,...,...,...,...,...,...,...,...,...
396107,KR_4296831363,2049.954102,7,34,4,1862.439209,"{'시공간': 2, '사이버네틱': 1, '수호자': 2, '반군': 1, '검사'...","{'TwistedFate': {'items': [58], 'star': 2}, 'X...",Platinum
396108,KR_4296831363,2049.954102,8,33,5,1799.791870,"{'시공간': 2, '사이버네틱': 1, '잠입자': 1, '마나 약탈자': 1, ...","{'Xayah': {'items': [5], 'star': 2}, 'Yasuo': ...",Platinum
396109,KR_4296831363,2049.954102,8,38,2,2041.694946,"{'총잡이': 1, '암흑의 별': 1, '메카 파일럿': 1, '용병': 1, '...","{'Poppy': {'items': [], 'star': 2}, 'Zoe': {'i...",Platinum
396110,KR_4296831363,2049.954102,8,35,3,1911.133057,"{'총잡이': 2, '시공간': 1, '폭파광': 1, '용병': 1, '반군': ...","{'Ziggs': {'items': [44, 48], 'star': 3}, 'Mal...",Platinum


In [258]:
# combination 재확인 

all_keys = set()
for comb_dict in df_all['combination']:
    all_keys.update(comb_dict.keys())

print(all_keys)

{'Soulbound', '암흑의 별', 'Ocean', '저격수', 'Mage', 'Mountain', '반군', 'Glacial', 'Light', 'Avatar', '별 수호자', 'Berserker', 'Desert', 'Metal', '시공간', 'Shadow', '사이버네틱', '잠입자', 'Summoner', '폭파광', 'Electric', '마법사', '마나 약탈자', 'Druid', '용병', '천상', '싸움꾼', '신비술사', 'Predator', '수호자', '메카 파일럿', '공허', 'Inferno', 'Crystal', '우주선', '우주 해적', 'Woodland', 'Assassin', 'Ranger', 'Wind', 'Alchemist', '선봉대', '검사', 'Poison', 'TemplateTrait', 'Warden', '발키리', '총잡이'}


In [259]:
# 왜 combination이 더 있지?
# 확인결과 -> 플래티넘,다이아몬드 매치에 시즌 2 데이터가 흘러들어옴.
# 추측: 모바일과 pc버전 업데이트 기간이 다르기 때문에 2시즌 데이터가 저장됐을 가능성 있음.
# 그렇다면 왜 플래티넘,다이아몬드 구간만 2시즌 데이터가 있을까? -> 모르겠음.

In [260]:
# 한글 매핑되지 않은 combination 이상치로 처리

def strange_comb(comb_dict):
    for k in comb_dict.keys():
        # 한글 매핑이 없는 항목은 combination_map에서 찾아볼 수 없으므로 'k' 그대로 남음
        if k not in combination_map.values():  # 한글 매핑된 값 목록에 없으면 이상치
            return None
    return comb_dict

In [261]:
# 한글 매핑되지 않은 combination 딕셔너리 행 삭제

df_all['combination'] = df_all['combination'].apply(strange_comb)
df_all = df_all.dropna(subset=['combination'])

In [262]:
# combination 재확인 (2)

all_keys = set()
for comb_dict in df_all['combination']:
    all_keys.update(comb_dict.keys())

print(all_keys)

{'암흑의 별', '저격수', '반군', '별 수호자', '시공간', '사이버네틱', '잠입자', '폭파광', '마법사', '마나 약탈자', '용병', '천상', '싸움꾼', '신비술사', '수호자', '메카 파일럿', '공허', '우주선', '우주 해적', '선봉대', '검사', '발키리', '총잡이'}


In [263]:
# combination때와 마찬가지로 문자열 딕셔너리로 변환

all_champs = set()
for champ_str in df_all['champion']:
    champ_dict = ast.literal_eval(champ_str) 
    all_champs.update(champ_dict.keys())

print(all_champs)

{'Neeko', 'Lulu', 'Graves', 'Sona', 'VelKoz', 'Jinx', 'Caitlyn', 'Ashe', 'Kassadin', 'Shaco', 'Syndra', 'Darius', 'Shen', 'Soraka', 'Zoe', 'Lucian', 'Leona', 'Irelia', 'Ezreal', 'Yasuo', 'Malphite', 'Blitzcrank', 'ChoGath', 'Ekko', 'MasterYi', 'Jayce', 'Gangplank', 'TwistedFate', 'Lux', 'Ahri', 'Poppy', 'Rumble', 'Vi', 'XinZhao', 'Xayah', 'WuKong', 'Fizz', 'Xerath', 'AurelionSol', 'KaiSa', 'Rakan', 'Fiora', 'Kayle', 'Karma', 'KhaZix', 'Annie', 'Ziggs', 'Mordekaiser', 'Jhin', 'Thresh', 'MissFortune', 'JarvanIV'}


In [264]:
champion_map = {
    "Graves": "그레이브즈",
    "Leona": "레오나",
    "Malphite": "말파이트",
    "Poppy": "뽀삐",
    "JarvanIV": "자르반 4세",
    "Xayah": "자야",
    "Zoe": "조이",
    "Ziggs": "직스",
    "KhaZix": "카직스",
    "Caitlyn": "케이틀린",
    "TwistedFate": "트위스티드 페이트",
    "Fiora": "피오라",
    "Darius": "다리우스",
    "Rakan": "라칸",
    "Lucian": "루시안",
    "Mordekaiser": "모데카이저",
    "Blitzcrank": "블리츠크랭크",
    "Sona": "소나",
    "Shen": "쉔",
    "XinZhao": "신 짜오",
    "Ahri": "아리",
    "Annie": "애니",
    "Yasuo": "야스오",
    "KaiSa": "카이사",
    "Neeko": "니코",
    "Lux": "럭스",
    "Rumble": "럼블",
    "MasterYi": "마스터 이",
    "Vi": "바이",
    "Shaco": "샤코",
    "Syndra": "신드라",
    "Ashe": "애쉬",
    "Ezreal": "이즈리얼",
    "Jayce": "제이스",
    "Karma": "카르마",
    "Kassadin": "카사딘",
    "VelKoz": "벨코즈",
    "Soraka": "소라카",
    "WuKong": "오공",
    "Irelia": "이렐리아",
    "Jhin": "진",
    "Jinx": "징크스",
    "ChoGath": "초가스",
    "Kayle": "케일",
    "Fizz": "피즈",
    "Gangplank": "갱플랭크",
    "Lulu": "룰루",
    "MissFortune": "미스 포츈",
    "Thresh": "쓰레쉬",
    "AurelionSol": "아우렐리온 솔",
    "Ekko": "에코",
    "Xerath": "제라스"
}

In [265]:
# champion 한글로 매핑

def champion_korean(champ_str):
    champ_dict = ast.literal_eval(champ_str)
    new_dict = {}
    for k, v in champ_dict.items():
        k_kr = champion_map.get(k, k) 
        new_dict[k_kr] = v
    return new_dict

df_all['champion'] = df_all['champion'].apply(champion_korean)

In [266]:
df_all

,gameId,gameDuration,level,lastRound,Ranked,ingameDuration,combination,champion,Tier
1,KR_4247538593,2142.470703,9,35,2,2134.272217,"{'총잡이': 2, '용병': 1, '반군': 6, '검사': 3, '싸움꾼': 1...","{'말파이트': {'items': [7], 'star': 2}, '야스오': {'i...",Challenger
2,KR_4247538593,2142.470703,8,34,3,2073.459229,"{'사이버네틱': 1, '암흑의 별': 3, '폭파광': 1, '잠입자': 4, '...","{'카이사': {'items': [99, 2, 23], 'star': 2}, '애니...",Challenger
4,KR_4247538593,2142.470703,9,33,5,1986.443237,"{'총잡이': 2, '폭파광': 2, '용병': 2, '반군': 6, '검사': 2...","{'직스': {'items': [], 'star': 1}, '야스오': {'item...",Challenger
5,KR_4247538593,2142.470703,8,31,6,1878.470093,"{'총잡이': 2, '폭파광': 2, '용병': 2, '반군': 6, '검사': 2...","{'직스': {'items': [], 'star': 1}, '말파이트': {'ite...",Challenger
6,KR_4247538593,2142.470703,9,31,7,1866.661499,"{'총잡이': 4, '시공간': 2, '사이버네틱': 2, '폭파광': 1, '용병...","{'말파이트': {'items': [], 'star': 2}, '그레이브즈': {'...",Challenger
...,...,...,...,...,...,...,...,...,...
396107,KR_4296831363,2049.954102,7,34,4,1862.439209,"{'시공간': 2, '사이버네틱': 1, '수호자': 2, '반군': 1, '검사'...","{'트위스티드 페이트': {'items': [58], 'star': 2}, '자야'...",Platinum
396108,KR_4296831363,2049.954102,8,33,5,1799.791870,"{'시공간': 2, '사이버네틱': 1, '잠입자': 1, '마나 약탈자': 1, ...","{'자야': {'items': [5], 'star': 2}, '야스오': {'ite...",Platinum
396109,KR_4296831363,2049.954102,8,38,2,2041.694946,"{'총잡이': 1, '암흑의 별': 1, '메카 파일럿': 1, '용병': 1, '...","{'뽀삐': {'items': [], 'star': 2}, '조이': {'items...",Platinum
396110,KR_4296831363,2049.954102,8,35,3,1911.133057,"{'총잡이': 2, '시공간': 1, '폭파광': 1, '용병': 1, '반군': ...","{'직스': {'items': [44, 48], 'star': 3}, '말파이트':...",Platinum


In [267]:
'''
할 일 :  아이템 번호->이름 매핑
         다양한 등수 별 시너지 (상위4위, 1위, 8위, ...)
         챔피언<->아이템 (어울리는 아이템) 관계 보여주기 
         다양한 그래프 사용해서 시각화
'''

'\n할 일 :  아이템 번호->이름 매핑\n         다양한 등수 별 시너지 (상위4위, 1위, 8위, ...)\n         챔피언<->아이템 (어울리는 아이템) 관계 보여주기 \n         다양한 그래프 사용해서 시각화\n'